In [1]:
from typing import Any, Dict, List, Optional,Set, Tuple 
import numpy as np
import pandas as pd
from random import sample,random
from math import floor
import matplotlib.pyplot as plt

def generate_data(size : int = 100,coefs :int = 4) -> np.array:
    """
    Untuk Dummy saja
    Generate Random data < Continous >
    size = How many observations
    coefs = How many features / how many chromosome
    Returns:
        np array (x), (y)
    """
    coeffs : List[float] = [0.4, 0.3 ,0.2, -0.1]
    x : List[np.array] = [[np.random.normal() for _ in range(len(coeffs))] for _ in range(size) ]
    y : List[np.array] = [np.dot(i,coeffs) for i in x]
    
    return np.array(x), np.array(y)



class Population:
    
    def __init__(self,TotalSize,Totalgenome: int) -> None:
        self.Totalgenome: int = Totalgenome
        self.totalSize : int = TotalSize
        self.bestIndividuals : List =[]
        self.population = []
        self.fitnessed  : Dict[str,Any]

    def createIndividu(self) -> np.array:
        """
        Create Individu using Real Numbers

        Returns:
            List[int]: Real numbers sequence
        """
        return np.random.normal(size=self.Totalgenome)
    
    def createPopulation(self) -> None :
        """
        Create Population with x size Depends on total size
        and append it to population list 
        
        Returns:
            None 
        """
        self.population.append([self.createIndividu() for _ in range(self.totalSize)])
   
    def fitness(self, individual , inputs : np.array, yTrue : np.array) -> Dict[str,Any]:
        """
        
        Fitness current Individual

        Args:
            individual (List[int]) --> Chromosome on a population
            inputs (np.array) ->> Input data
            yTrue (np.array) --> Groud truth
        Returns:
            Dict[str,Any]: [description]
        """
        info : Dict[str,Any] = {}
        bias : np.float = 0
        predicted : np.array =  bias + np.dot(inputs,individual) 
        yTrue_mean : np.float = np.mean(yTrue) 

        SST : np.float = np.sum(np.array([(y - yTrue_mean) ** 2 for y in yTrue]),axis=None)
        SSR : np.float = np.sum(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)]),axis = None)
    
        bias :np.float = np.mean(np.sqrt(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)])))
        RMSE : np.float = np.sqrt(SSR/ len(y))
        Rsquared : np.float = (1 - (SSR / SST))

        SSE : np.float = SSR / len(y)
        oneDivError : np.float = 1 / SSE

        info["Rsquared"] = Rsquared
        info["coeff"] = individual
        info["error"] = SSE
        info["RMSE"] = RMSE 
        info["divByOne"] = oneDivError
        info["bias"] = bias 
        self.fitnessed = info
        return info
    
    def evaluate_population(self, x:np.array ,y:np.array ,selectionSize : int) -> None :
        """
        Function to evaluate the best individual from current population

        Args:
           x (np.array) , Input data
           y (np.array, Ground truth
           selectionSize (int) : How many best Individuals you want to grab from the population

        Returns:
            None
        """
        fitness_list= [self.fitness(individual,x,y) for individual in self.population[0]]
        error_list : List[Dict[str,Any]] = sorted(fitness_list,key=lambda i : i["error"])
        best_individuals = error_list[: selectionSize]
        self.bestIndividuals.clear()
        self.bestIndividuals.append(best_individuals)
        
        print(f"Error {best_individuals[0]['error']}\n RSquared {best_individuals[0]['Rsquared']}\n") 


    def mutate(self,individual : List[float], probabilityMutating : float) -> List[float]:
        """
        
        Ini Fungsinya lakuin mutasi 

        Args:
            individual (List[float]): Individu pada populasi
            probabilityMutating (float): Probabilitas Individu saat ini bermutasi 

        Returns:
            List[float]: Individu Yang telah bermutasi < Jika dilakukan >
        """
        indx : List[int] = [i for i in range(len(individual))]

        totalMutatedGens : int = int(probabilityMutating * len(individual))
        indx_toMutate : List[int] = sample(indx,k = totalMutatedGens)
        for ind in indx_toMutate:
            choice : np.int = np.random.choice([-1,1])
            gene_transform : float = choice*random()

            individual[ind] = individual[ind] + gene_transform
        return individual
        
    def crossover(self, parent1 : Dict[str,Any], parent2 : Dict[str,Any]) -> None:
        """
           CrossOver / persilangan untuk menghasilkan keturunan 
           metode yang digunakan adalah Simple Arithmetic crossover
        Args:
            parent1 (Dict[str,Any]): [description]
            parent2 (Dict[str,Any]): [description]
        Return: None
        """
        
        anak_keturunan : Dict[int,Any] = {}
        index : List[int] = [i for i in range( self.Totalgenome )]
        indexRandomize : List[int] = sample(index, floor(0.5 * self.Totalgenome))
        IndexNotInRandomize : List[int] = [i for i in index if i not in indexRandomize]

        getCromosomeFromParent1 : List[Any] = [[i,parent1['coeff'][i]] for i in indexRandomize]
        getCromosomeFromParent2 : List[Any] = [[i,parent2["coeff"][i]] for i in IndexNotInRandomize]
        
        anak_keturunan.update({key :value for (key,value) in getCromosomeFromParent1})
        anak_keturunan.update({key : value for(key,value) in getCromosomeFromParent2})
    
        return [anak_keturunan[i] for i in index]

    def create_new_generation(self,probabilityMutating : float,probability_indiv_mutating:float) -> None:
        """
        Create new population using the best individuals

        Args:
            probabilityMutating(float) : 
            probability_indiv_mutating (float) : 

        Return:
            None
        """
        pasangan_sah = [sample(self.bestIndividuals[0],2) for _ in range( self.totalSize)]
        crossOverered_parents = [self.crossover(pasangan[0],pasangan[1]) for pasangan in pasangan_sah]
        pasangan_sah_indx = [i for i in range(self.totalSize)]
        pasanganCalonMutasi = sample(pasangan_sah_indx,floor(probabilityMutating * self.totalSize))

        PasanganMutasi = [[i,self.mutate(crossOverered_parents[i],probability_indiv_mutating)] for i in pasanganCalonMutasi]
        for anakMutasi in PasanganMutasi:
            crossOverered_parents[anakMutasi[0]] = anakMutasi[1]
        self.population.clear()
        self.population.append(crossOverered_parents)
    
if __name__ == "__main__":
    x,y = generate_data(100)
    np.random.seed(123) 
    pop =Population(100,4)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.8
    probability_indiv_mutation = 0.25

    for i in range(3) :
        print(f"generation : {i}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        
    
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])
    

    #parent1 = pop.fitness(individual,x,y)
    #parent2 = pop.fitness(individual,x,y)

generation : 0
Error 0.48159763989164
 RSquared -0.6134593972123599

generation : 1
Error 0.012328996020356232
 RSquared 0.9586951163387899

generation : 2
Error 0.005170153219379656
 RSquared 0.9826788347660658

Nilai x :  [-1.00949851  0.18782672 -2.13888721  0.52588276]
Nilai Y :  -0.8278171072573787
Predicted =  -0.7145776857889589
{'Rsquared': 0.9826788347660658, 'coeff': [0.42598252748922666, 0.2786835167270544, 0.1771923291989879, -0.02996800704754226], 'error': 0.005170153219379656, 'RMSE': 0.07190377750424282, 'divByOne': 193.41786549992915, 'bias': 0.057860896151107385}


In [2]:
def terminate(ans : Dict[str,Any]):
    """
    Terminate Genetic Algorithm Menggunakan nilai RSquared yang telah didefinisiskan
    
    ans : Dict - > Berisi nilai Rsquared,coeff,RMSE ...
    
    return 
    True jika Rsquared > 0.95
    False others
    """
    if ans["Rsquared"] > 0.95:
        return True
    else :
        return False

In [3]:
from sklearn.preprocessing import StandardScaler
StdScaler = StandardScaler()


In [4]:
"""
Codingan Ini digunakan untuk preprocessing data saham dari link berikut : https://github.com/wildangunawan/Dataset-Saham-IDX
menjadi nilai open selama 10 hari berturut turut

df = pd.read_csv("task3.csv")
dfnew = pd.DataFrame()
yow = []
def generate_newdataset():
    k=13
    j=1
    for i in range(100):
        
        dict1 = {}
        dict1.update( {i:df[j:k]["open_price"].values})
        j+=1
        k+=1
        yow.append(dict1)
generate_newdataset()
dfnew = pd.DataFrame(yow[0])
for i in range(1,len(yow)):
    dfnew[i] = pd.DataFrame(yow[i])
dfnew = dfnew.T
dfnew.to_csv("dataset.csv")
"""

'\nCodingan Ini digunakan untuk preprocessing data saham dari link berikut : https://github.com/wildangunawan/Dataset-Saham-IDX\nmenjadi nilai open selama 10 hari berturut turut\n\ndf = pd.read_csv("task3.csv")\ndfnew = pd.DataFrame()\nyow = []\ndef generate_newdataset():\n    k=13\n    j=1\n    for i in range(100):\n        \n        dict1 = {}\n        dict1.update( {i:df[j:k]["open_price"].values})\n        j+=1\n        k+=1\n        yow.append(dict1)\ngenerate_newdataset()\ndfnew = pd.DataFrame(yow[0])\nfor i in range(1,len(yow)):\n    dfnew[i] = pd.DataFrame(yow[i])\ndfnew = dfnew.T\ndfnew.to_csv("dataset.csv")\n'

In [5]:
if __name__ == "__main__":
    df = pd.read_csv("dataset.csv")
    df = df.iloc[:,1:13]
    x = df.iloc[:,1:12]
    y = df.iloc[:,11]
    
    y = StdScaler.fit_transform(np.array(y).reshape(-1,1))
    x = StdScaler.fit_transform(x) 
    np.random.seed(123) 
    pop =Population(100,11)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.2
    probability_indiv_mutation = 0.25

    terminate_ga = False
    generation = 0
    while(terminate_ga != True): 
        print(f"generation : {generation}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        terminate_ga = terminate(pop.fitnessed)
        generation += 1
        
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])

generation : 0
Error 1.0696958324096997
 RSquared -0.06969583240969945

generation : 1
Error 0.6290681894801432
 RSquared 0.370931810519857

generation : 2
Error 0.16249791402772853
 RSquared 0.8375020859722715

generation : 3
Error 0.16249791402772853
 RSquared 0.8375020859722715

generation : 4
Error 0.09107568027816156
 RSquared 0.9089243197218385

generation : 5
Error 0.07001455269170775
 RSquared 0.9299854473082922

generation : 6
Error 0.06289168754935259
 RSquared 0.9371083124506474

generation : 7
Error 0.04290367803501316
 RSquared 0.9570963219649868

generation : 8
Error 0.04290367803501316
 RSquared 0.9570963219649868

generation : 9
Error 0.04290367803501316
 RSquared 0.9570963219649868

generation : 10
Error 0.04290367803501316
 RSquared 0.9570963219649868

Nilai x :  [2.75086792 2.86571695 2.32914034 1.99572509 0.87738216 0.74882796
 1.29885108 1.43537095 1.41540623 1.54884825 1.37563292]
Nilai Y :  [1.37563292]
Predicted =  1.4708164459201218
{'Rsquared': 0.9570963219649

In [6]:

predicted = pop.bestIndividuals[0][0]['bias'] + np.dot(x, pop.bestIndividuals[0][0]["coeff"])
predicted

array([ 1.47081645,  1.10057325,  1.30271369,  1.1284171 ,  1.19727185,
        1.23022564,  0.98722914,  0.52281403,  0.35322617,  0.19856693,
       -0.13075305, -0.10253515,  0.23561514,  0.64199228,  0.54156576,
        0.17381888, -0.07825348, -0.29265991,  0.04344385, -0.07815723,
       -0.02411523,  0.1421636 ,  0.68435145,  0.63498224, -0.00402645,
       -0.34135808,  0.04850221,  0.67614753,  0.36550823, -0.3268243 ,
       -0.4018213 , -0.16457961,  0.05803848, -0.3147975 , -0.34595372,
       -0.60375602, -1.25899413, -1.67148908, -2.10617937, -2.29855176,
       -1.88610155, -1.76578729, -1.88134856, -1.4625198 , -1.17463053,
       -1.40214645, -1.28837362, -1.14691083, -0.88459431, -0.39893248,
       -0.55449317,  0.0932173 ,  0.65519555,  0.40972688,  0.50390496,
       -0.00365355, -0.13755987,  0.17124353,  0.21672368, -0.53182809,
        0.2697137 ,  0.23301209,  0.02183307,  0.04966869, -0.12046145,
       -0.13240478, -0.14818368, -0.40952363, -0.58073809,  0.12

In [7]:
%whos

Variable                     Type                          Data/Info
--------------------------------------------------------------------
Any                          _Any                          typing.Any
Dict                         GenericMeta                   typing.Dict
List                         GenericMeta                   typing.List
Optional                     _Optional                     typing.Optional
Population                   type                          <class '__main__.Population'>
Set                          GenericMeta                   typing.Set
StandardScaler               type                          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
StdScaler                    StandardScaler                StandardScaler()
Tuple                        TupleMeta                     typing.Tuple
df                           DataFrame                              0       1       <...>\n[100 rows x 12 columns]
floor                        builtin_f